# 01 — Collect & Clean

In [1]:
# Imports
import pandas as pd
import numpy as np
import re
from pathlib import Path

pd.set_option('display.max_columns', 100)
DATA_DIR = Path('data')
RAW_DIR = DATA_DIR / 'raw'
INTERIM_DIR = DATA_DIR / 'interim'
INTERIM_DIR.mkdir(parents=True, exist_ok=True)

## 1) Load raw data

In [11]:
# Example: load raw listings
# Change the filename to your actual raw CSV
raw_path = Path("..") / RAW_DIR / 'toronto_apartments_raw.csv'
if not raw_path.exists():
    print('⚠️ Raw file not found:', raw_path)

# If the file exists, load it
if raw_path.exists():
    df = pd.read_csv(raw_path)
else:
    # fallback empty frame with expected columns to let notebook run
    df = pd.DataFrame(columns=['title','price','sqft','bedrooms','bathrooms','address','neighbourhood','url','source'])

print(df.shape)
df.head(3)

(101, 8)


,Unnamed: 0,price,size,address,bedrooms,bathrooms,furnished,apartmentOrCondo
0,https://rentals.ca/toronto/35-mariner-harbourv...,2995,750,"35 Mariner Terr - Toronto, ON",1.5,1.0,0,apartment
1,https://rentals.ca/toronto/224-king-street-wes...,2800,600,"224 King Street West - Toronto, ON",1.0,1.0,0,condo
2,https://rentals.ca/toronto/3216-11-brunel-ct-t...,2095,500,"11 Brunel Court - Toronto, ON",1.0,1.0,0,apartment


## 2) Standardize column names

In [12]:
# map common variations -> canonical names
rename_map = {
    'Price':'price', 'Rent':'price',
    'Sqft':'sqft', 'Area':'sqft', 'Size':'sqft',
    'Beds':'bedrooms','bed':'bedrooms','Bedrooms':'bedrooms',
    'Baths':'bathrooms','Bathrooms':'bathrooms',
    'Address':'address','Location':'address','Addr':'address',
    'Neighborhood':'neighbourhood','Neighbourhood':'neighbourhood',
    'URL':'url','Link':'url'
}

# lower-case, strip, replace spaces with underscores
clean_cols = {c: re.sub(r'\s+', '_', c.strip().lower()) for c in df.columns}
df = df.rename(columns=clean_cols)
# second pass for common variants
df = df.rename(columns={k.lower(): v for k, v in rename_map.items() if k.lower() in df.columns})

expected = ['title','price','sqft','bedrooms','bathrooms','address','neighbourhood','url','source']
for col in expected:
    if col not in df.columns:
        df[col] = np.nan

df = df[expected]
df.head(3)

,title,price,sqft,bedrooms,bathrooms,address,neighbourhood,url,source
0,NaN,2995,750,1.5,1.0,"35 Mariner Terr - Toronto, ON",NaN,NaN,NaN
1,NaN,2800,600,1.0,1.0,"224 King Street West - Toronto, ON",NaN,NaN,NaN
2,NaN,2095,500,1.0,1.0,"11 Brunel Court - Toronto, ON",NaN,NaN,NaN


## 3) Basic cleaning helpers

In [13]:
def parse_price(val):
    if pd.isna(val):
        return np.nan
    s = str(val)
    # keep digits only
    digits = re.sub(r'[^0-9]', '', s)
    if digits == '':
        return np.nan
    return float(digits)

# принимает '650–700 sqft' или '650 sqft' -> 650
# принимает '—'/'na' -> NaN
def parse_sqft(val):
    if pd.isna(val):
        return np.nan
    s = str(val).lower()
    # replace range with the first number
    m = re.findall(r'(\d+(?:\.\d+)*)', s)
    if not m:
        return np.nan
    return float(m[0])

# '1 bed', '1+den', 'studio' -> 0 for studio, else number

def parse_bedrooms(val):
    if pd.isna(val):
        return np.nan
    s = str(val).lower()
    if 'studio' in s:
        return 0
    m = re.search(r'(\d+(?:\.\d+)*)', s)
    return float(m.group(1)) if m else np.nan

# '1 bath', '1.5 baths' -> float

def parse_bathrooms(val):
    if pd.isna(val):
        return np.nan
    s = str(val).lower()
    m = re.search(r'(\d+(?:\.\d+)*)', s)
    return float(m.group(1)) if m else np.nan

# normalize addresses (без геокодинга)

def clean_address(val):
    if pd.isna(val):
        return np.nan
    s = str(val).strip()
    s = re.sub(r'\s+', ' ', s)
    return s

## 4) Apply cleaning

In [14]:
df['price'] = df['price'].apply(parse_price).astype('float')
df['sqft'] = df['sqft'].apply(parse_sqft).astype('float')
df['bedrooms'] = df['bedrooms'].apply(parse_bedrooms).astype('float')
df['bathrooms'] = df['bathrooms'].apply(parse_bathrooms).astype('float')
df['address'] = df['address'].apply(clean_address)

# optional: clip implausible values before IQR to reduce influence
# df['price'] = df['price'].clip(lower=600, upper=12000)
# df['sqft'] = df['sqft'].clip(lower=150, upper=3000)

print(df.dtypes)
df.head(5)

title            float64
price            float64
sqft             float64
bedrooms         float64
bathrooms        float64
address           object
neighbourhood    float64
url              float64
source           float64
dtype: object


,title,price,sqft,bedrooms,bathrooms,address,neighbourhood,url,source
0,NaN,2995.0,750.0,1.5,1.0,"35 Mariner Terr - Toronto, ON",NaN,NaN,NaN
1,NaN,2800.0,600.0,1.0,1.0,"224 King Street West - Toronto, ON",NaN,NaN,NaN
2,NaN,2095.0,500.0,1.0,1.0,"11 Brunel Court - Toronto, ON",NaN,NaN,NaN
3,NaN,4200.0,1200.0,2.0,2.0,"85 Queens Wharf Road - Toronto, ON",NaN,NaN,NaN
4,NaN,2300.0,600.0,1.0,1.0,"81 Navy Wharf Court - Toronto, ON",NaN,NaN,NaN


## 5) Drop exact duplicates

In [15]:
import re
from urllib.parse import urlparse

def canonical_url(u: str) -> str:
    if pd.isna(u) or not str(u).strip():
        return pd.NA
    p = urlparse(str(u).strip())
    # домен + путь, без query/fragment — чтобы одна страница считалась одной
    return f"{p.scheme}://{p.netloc}{p.path}".rstrip('/')

def norm_address(a: str) -> str:
    if pd.isna(a):
        return ""
    s = str(a).lower()
    s = re.sub(r'[\.,#]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

# грубый парсер юнита из address/title (ищем apt/unit/suite/# и формат '123-45 Some St')
UNIT_PAT = re.compile(r'\b(apt|unit|suite|apartment)\s*([a-z0-9\-]+)\b|#\s*([a-z0-9\-]+)|^(\d+)\s*-\s*\d+', re.I)

def extract_unit(text1: str, text2: str=""):
    for s in (text1 or "", text2 or ""):
        m = UNIT_PAT.search(str(s))
        if m:
            # вернём первое непустое из групп
            for g in m.groups():
                if g:
                    return str(g).lower()
    return ""

def approx_equal(a, b, rel=0.05, abs_tol=25):
    """Считаем одинаковыми, если разница <5% или <25$ (или <5%/25 sqft)."""
    if pd.isna(a) or pd.isna(b):
        return False
    a, b = float(a), float(b)
    return abs(a-b) <= max(abs(a)*rel, abs_tol)

# 1) Подготовим служебные колонки
df["url_canon"] = df["url"].apply(canonical_url) if "url" in df.columns else pd.NA
df["address_norm"] = df["address"].apply(norm_address)
df["unit"] = [extract_unit(a, t) for a, t in zip(df.get("address",""), df.get("title",""))]

# 2) Сначала уберём настоящие копии (полные дубли строк)
before = len(df)
df = df.drop_duplicates()
print(f"Exact row duplicates removed: {before - len(df)}")

# 3) Жёсткие дубли по фиксированному набору признаков
key_cols = ["address_norm", "unit", "bedrooms", "bathrooms", "sqft", "price", "url_canon"]
before = len(df)
df = df.drop_duplicates(subset=[c for c in key_cols if c in df.columns], keep="first")
print(f"Strict duplicates removed: {before - len(df)}")

# 4) Гибкая дедупликация ВНУТРИ одного url_canon:
#    Если address_norm и unit совпадают, а (price И sqft И bedrooms И bathrooms) примерно равны — считаем дублем.
def dedupe_within_url(group: pd.DataFrame) -> pd.DataFrame:
    keep_idx = []
    for idx, row in group.iterrows():
        same = False
        for k in keep_idx:
            r = group.loc[k]
            if (row["address_norm"] == r["address_norm"]) and (row["unit"] == r["unit"]):
                cond_price = approx_equal(row.get("price"), r.get("price"))
                cond_sqft  = approx_equal(row.get("sqft"), r.get("sqft"))
                cond_bed   = approx_equal(row.get("bedrooms"), r.get("bedrooms"), rel=0, abs_tol=0.01)
                cond_bath  = approx_equal(row.get("bathrooms"), r.get("bathrooms"), rel=0, abs_tol=0.01)
                if cond_price and cond_sqft and cond_bed and cond_bath:
                    same = True
                    break
        if not same:
            keep_idx.append(idx)
    return group.loc[keep_idx]

before = len(df)
if "url_canon" in df.columns:
    df = (df.groupby("url_canon", dropna=False, group_keys=False)
            .apply(dedupe_within_url)
            .reset_index(drop=True))
print(f"Near-duplicates within same URL removed: {before - len(df)}")

# 5) Дедупликация МЕЖДУ разными URL для одного и того же адреса/юнита:
#    Если адрес/юнит совпадают и признаки примерно равны — оставим первую запись (например, из «предпочтительного» источника).
pref_source_order = ["rentals.ca", "condos.ca", "realtor.ca"]  # примеры; под себя
def source_rank(u):
    if pd.isna(u): return 999
    for i, dom in enumerate(pref_source_order):
        if dom in u:
            return i
    return 100

df["source_rank"] = df["url_canon"].apply(source_rank)

def dedupe_across_urls(group: pd.DataFrame) -> pd.DataFrame:
    group = group.sort_values("source_rank")
    keep_idx = []
    for idx, row in group.iterrows():
        same = False
        for k in keep_idx:
            r = group.loc[k]
            cond_price = approx_equal(row.get("price"), r.get("price"))
            cond_sqft  = approx_equal(row.get("sqft"), r.get("sqft"))
            cond_bed   = approx_equal(row.get("bedrooms"), r.get("bedrooms"), rel=0, abs_tol=0.01)
            cond_bath  = approx_equal(row.get("bathrooms"), r.get("bathrooms"), rel=0, abs_tol=0.01)
            if cond_price and cond_sqft and cond_bed and cond_bath:
                same = True
                break
        if not same:
            keep_idx.append(idx)
    return group.loc[keep_idx]

before = len(df)
df = (df.groupby(["address_norm","unit"], dropna=False, group_keys=False)
        .apply(dedupe_across_urls)
        .reset_index(drop=True))
print(f"Cross-URL near-duplicates removed: {before - len(df)}")

# (опционально) идентификатор варианта — поможет в аналитике
import hashlib
def variant_id(row):
    key = f"{row['address_norm']}|{row['unit']}|{row.get('bedrooms')}|{row.get('bathrooms')}|{round(row.get('sqft') or 0)}|{round(row.get('price') or 0)}"
    return hashlib.sha1(key.encode()).hexdigest()[:10]

df["variant_id"] = df.apply(variant_id, axis=1)


Exact row duplicates removed: 2
Strict duplicates removed: 0
Near-duplicates within same URL removed: 0
Cross-URL near-duplicates removed: 0


C:\Users\zhant\AppData\Local\Temp\ipykernel_2280\552793985.py:78: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(dedupe_within_url)
C:\Users\zhant\AppData\Local\Temp\ipykernel_2280\552793985.py:114: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(dedupe_across_urls)


## 6) Handle missing values

In [16]:
# Критичные поля для модели — price и (address или url)
crit = df['price'].notna() & (df['address'].notna() | df['url'].notna())
df = df[crit].copy()

# Не критичные — можно оставить NaN или имPUTe позже
# Например, bedrooms/bathrooms иногда NaN — пусть остаются

print(df.isna().mean().sort_values(ascending=False).head(10))
print(df.shape)


title            1.0
neighbourhood    1.0
url_canon        1.0
source           1.0
url              1.0
price            0.0
address          0.0
bathrooms        0.0
bedrooms         0.0
sqft             0.0
dtype: float64
(99, 14)


## 7) Remove outliers with IQR 

In [17]:
def iqr_filter(series, k=1.5):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lo = q1 - k*iqr
    hi = q3 + k*iqr
    return (series >= lo) & (series <= hi)

mask = iqr_filter(df['price'])
if df['sqft'].notna().any():
    mask &= iqr_filter(df['sqft'])

before = len(df)
df = df[mask].copy()
print(f"Removed outliers: {before - len(df)} | New size: {len(df)}")

Removed outliers: 10 | New size: 89


## 8) Save cleaned dataset

In [18]:
out_path = Path("..") / INTERIM_DIR / 'clean_listings.csv'
df.to_csv(out_path, index=False)
print('Saved:', out_path)
df.head(3)

Saved: ..\data\interim\clean_listings.csv


,title,price,sqft,bedrooms,bathrooms,address,neighbourhood,url,source,url_canon,address_norm,unit,source_rank,variant_id
0,NaN,2995.0,750.0,1.5,1.0,"35 Mariner Terr - Toronto, ON",NaN,NaN,NaN,<NA>,35 mariner terr - toronto on,,999,69fbb2161b
1,NaN,2800.0,600.0,1.0,1.0,"224 King Street West - Toronto, ON",NaN,NaN,NaN,<NA>,224 king street west - toronto on,,999,7140165d6e
2,NaN,2095.0,500.0,1.0,1.0,"11 Brunel Court - Toronto, ON",NaN,NaN,NaN,<NA>,11 brunel court - toronto on,,999,fd258ce795
